# Initialization

In [52]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import os
import logging
import pdb
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import precision_recall_curve,precision_score,recall_score,confusion_matrix


from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/autoencoder_example'
model_dir = '/content/drive/My Drive/autoencoder_example/autoencoder_model_v1'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data

In [0]:
df = pd.read_pickle(os.path.join(root_dir,'creditcard.pkl'),compression='gzip')
full_y = df.Class
full_X = df[[col for col in df.columns if col != 'Class']]

# CV

In [0]:
seed = 0
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
ksplit_indexes = list(kfold.split(full_X,full_y))

In [0]:
with open(os.path.join(root_dir,'kfold.pkl'),'wb') as f:
    pickle.dump({
        'kfold':kfold,
        'ksplit_indexes':ksplit_indexes
    },f)
    
with open(os.path.join(root_dir,'kfold.pkl'),'rb') as f:
    pickled = pickle.load(f)
    kfold = pickled['kfold']
    ksplit_indexes = pickled['ksplit_indexes']

# CV part2

In [0]:
ind = 0

train_inds, test_inds = ksplit_indexes[ind]

train_X = full_X.iloc[train_inds].drop(['Time'],axis=1)
train_y = full_y.iloc[train_inds]

normal_train_mask = train_y[train_y == 0].index


test_X = full_X.iloc[test_inds].drop(['Time'],axis=1)
test_y = full_y.iloc[test_inds]

normal_test_mask = test_y[test_y == 0].index

sc = MinMaxScaler().fit(train_X.loc[normal_train_mask])

# AutoEncoder Estimator

In [0]:
class AutoEncoderEstimator:
    def __init__(self,params,model_dir):
        self.model_dir = model_dir
        self.params = params.copy()
        self.params['model_dir'] = model_dir
#         pdb.set_trace()
        self.estimator = tf.estimator.Estimator(model_fn=self._model_func,model_dir=self.model_dir,params=params)
        with open(os.path.join(self.model_dir,'params.pkl') , 'wb') as f:
            pickle.dump(self.params,f)
    

    def train(self,X_df,batch_size,steps):
        self.estimator.train(input_fn=lambda:self._train_input_fn((X_df,),(X_df,),batch_size),steps=steps)
        print('finish training')
        
    def evaluate(self,X_df,batch_size):
        self.estimator.evaluate(input_fn=lambda:self._eval_input_fn((X_df,),(X_df,),batch_size))
        print('finish evaluation') 
        
    def predict(self,X_df,batch_size):
        return self.estimator.predict(input_fn=lambda:self._eval_input_fn((X_df,),(X_df,),batch_size))
        
        
    ####################wrapp rnn as estimator
    def _train_input_fn(self,features, labels, batch_size):
        """An input function for training"""
        # Convert the inputs to a Dataset.
        dataset = tf.data.Dataset.from_tensor_slices( (features, labels) )
        dataset = dataset.repeat().batch(batch_size)#.shuffle(1000).repeat().batch(batch_size)
        return dataset


    def _eval_input_fn(self,features, labels, batch_size):
        """An input function for evaluation or prediction"""
    #     features=dict(features)
        if labels is None:
            # No labels, use only features.
            inputs = (features,features)
        else:
            inputs = (features, labels)

        # Convert the inputs to a Dataset.
        dataset = tf.data.Dataset.from_tensor_slices(inputs)

        # Batch the examples
        assert batch_size is not None, "batch_size must not be None"
        dataset = dataset.batch(batch_size)

        # Return the dataset.
        return dataset
    
    def _model_func(self,features, labels, mode, params):
        ###########params
        input_len = params['input_len']
        hidden_layer_sizes = params['hidden_layer_sizes']
        hidden_layer_activations_func = params['hidden_layer_activations_func']
        learning_rate = params['learning_rate']
        
        ###########define input
        X = tf.cast(features[0], tf.float32)

        ###########define hidden layers
        weights = []
        biases = []
        Zs = [X]

        for layer_ind,s in enumerate(hidden_layer_sizes):
            act_func_key = hidden_layer_activations_func[layer_ind]
            if layer_ind == 0:
                prev_input_size = input_len
            else:
                prev_input_size = hidden_layer_sizes[layer_ind - 1]

            W = tf.Variable(tf.truncated_normal([prev_input_size, s], -0.1, 0.1),dtype=tf.float32)
            b = tf.Variable(tf.truncated_normal([s,], -0.1, 0.1),dtype=tf.float32)

            Z = act_func_key( tf.matmul( Zs[-1] ,W ) + b )

            weights.append(W)
            biases.append(b)

            Zs.append(Z)

        ######output layer weight
        W = tf.Variable(tf.truncated_normal([hidden_layer_sizes[-1], input_len], -0.1, 0.1),dtype=tf.float32)
        b = tf.Variable(tf.truncated_normal([input_len], -0.1, 0.1),dtype=tf.float32)
        Z = tf.matmul( Zs[-1] ,W ) + b

        ######mse
        difference = Z - X
        mse = tf.reduce_mean(difference * difference,axis=1)####for prediction
        
    
        if mode == tf.estimator.ModeKeys.PREDICT:
            prediction_bundle = {
                'mse': mse,
                'Z':Z
            }
            return tf.estimator.EstimatorSpec(mode, predictions=prediction_bundle)            

        ######loss
        total_loss = tf.reduce_sum(mse)        
        
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(mode, loss=total_loss)        
        

        # Create training op.
        assert mode == tf.estimator.ModeKeys.TRAIN
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(total_loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=total_loss, train_op=train_op)        
        
        

In [21]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(os.path.join(root_dir,'log'))
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)


input_len = len(train_X.columns)
hidden_layer_sizes = [16]
hidden_layer_activations_func = [tf.nn.sigmoid]
learning_rate = 0.0001

params = {
    'input_len':input_len,
    'hidden_layer_sizes':hidden_layer_sizes,
    'hidden_layer_activations_func':hidden_layer_activations_func,
    'learning_rate':learning_rate,
}

ae = AutoEncoderEstimator(params,model_dir)

I0731 16:12:31.595314 139869363488640 estimator.py:1790] Using default config.
I0731 16:12:31.603216 139869363488640 estimator.py:209] Using config: {'_model_dir': '/content/drive/My Drive/autoencoder_example/autoencoder_model_v1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f35978fc1d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas

In [0]:
for i in range(50):
    ae.train(sc.transform(train_X.loc[normal_train_mask]),batch_size=20000,steps=300)
    ae.evaluate(sc.transform(test_X.loc[normal_test_mask]),batch_size=20000)

I0731 17:08:21.005395 139869363488640 estimator.py:1145] Calling model_fn.
I0731 17:08:21.174324 139869363488640 estimator.py:1147] Done calling model_fn.
I0731 17:08:21.180253 139869363488640 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0731 17:08:21.287770 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:08:21.300919 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-12000
I0731 17:08:21.373402 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:08:21.385728 139869363488640 session_manager.py:502] Done running local_init_op.
I0731 17:08:49.232966 139869363488640 basic_session_run_hooks.py:606] Saving checkpoints for 12000 into /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt.
I0731 17:08:50.860307 139869363488640 basic_session_run_hooks.py:262] loss = 46.341076, step = 12001
I0731 17:08:57.880023 139869363488640 basic_

finish training


I0731 17:09:12.359978 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:09:12.367150 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-12300
I0731 17:09:12.398751 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:09:12.411534 139869363488640 session_manager.py:502] Done running local_init_op.
I0731 17:09:12.735035 139869363488640 evaluation.py:275] Finished evaluation at 2019-07-31-17:09:12
I0731 17:09:12.737097 139869363488640 estimator.py:2039] Saving dict for global step 12300: global_step = 12300, loss = 34.251686
I0731 17:09:12.744577 139869363488640 estimator.py:2099] Saving 'checkpoint_path' summary for global step 12300: /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-12300


finish evaluation


I0731 17:09:13.144878 139869363488640 estimator.py:1145] Calling model_fn.
I0731 17:09:13.315037 139869363488640 estimator.py:1147] Done calling model_fn.
I0731 17:09:13.320462 139869363488640 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0731 17:09:13.431852 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:09:13.442148 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-12300
I0731 17:09:13.511185 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:09:13.524812 139869363488640 session_manager.py:502] Done running local_init_op.
I0731 17:09:41.256057 139869363488640 basic_session_run_hooks.py:606] Saving checkpoints for 12300 into /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt.
I0731 17:09:43.045415 139869363488640 basic_session_run_hooks.py:262] loss = 46.164467, step = 12301
I0731 17:09:49.636748 139869363488640 basic_

...

I0731 17:29:18.506091 139869363488640 estimator.py:1145] Calling model_fn.
I0731 17:29:18.674412 139869363488640 estimator.py:1147] Done calling model_fn.
I0731 17:29:18.678725 139869363488640 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0731 17:29:18.788205 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:29:18.800967 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-19200
I0731 17:29:18.865660 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:29:18.877032 139869363488640 session_manager.py:502] Done running local_init_op.
I0731 17:29:46.413054 139869363488640 basic_session_run_hooks.py:606] Saving checkpoints for 19200 into /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt.
I0731 17:29:49.613010 139869363488640 basic_session_run_hooks.py:262] loss = 33.70906, step = 19201
I0731 17:29:56.450500 139869363488640 basic_s

finish training


I0731 17:30:10.819940 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:30:10.829057 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-19500
I0731 17:30:10.865365 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:30:10.878200 139869363488640 session_manager.py:502] Done running local_init_op.
I0731 17:30:11.191250 139869363488640 evaluation.py:275] Finished evaluation at 2019-07-31-17:30:11
I0731 17:30:11.193205 139869363488640 estimator.py:2039] Saving dict for global step 19500: global_step = 19500, loss = 22.690826
I0731 17:30:11.201176 139869363488640 estimator.py:2099] Saving 'checkpoint_path' summary for global step 19500: /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-19500


finish evaluation


I0731 17:30:11.584583 139869363488640 estimator.py:1145] Calling model_fn.
I0731 17:30:11.748768 139869363488640 estimator.py:1147] Done calling model_fn.
I0731 17:30:11.752959 139869363488640 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0731 17:30:11.868121 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:30:11.878730 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-19500
I0731 17:30:11.943085 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:30:11.955205 139869363488640 session_manager.py:502] Done running local_init_op.


# Tuning Threshold

In [64]:
train_pred = ae.predict(sc.transform(train_X),batch_size=20000)

pred_mse = []
pred_X_clone = []

for tp in train_pred:
  pred_mse.append( tp['mse'] )
  pred_X_clone.append( tp['Z'] )

train_pred_X_clone = np.array(pred_X_clone)

tune_df = pd.DataFrame({'mse':pred_mse,'y':train_y.tolist()})

precisions, recalls, thresholds = precision_recall_curve(y_true=tune_df.y,probas_pred=tune_df.mse)

precisions = precisions[:-1]
recalls = recalls[:-1]

f1 = 2 * precisions * recalls / (recalls + precisions)

threshold_df = pd.DataFrame({'precision':precisions,'recall':recalls,'f1':f1},index=thresholds)


########selecting best thres
best_thres = threshold_df.f1.idxmax()
# best_thres = threshold_df.recall.idxmax()
# best_thres = threshold_df.precision.idxmax()

#######threshold evaluation on training
tune_df['y_pred'] = (tune_df.mse >= best_thres).astype('int')
print('best mse threshold = ',best_thres)
print( precision_score(y_true=tune_df.y,y_pred=tune_df.y_pred),recall_score(y_true=tune_df.y,y_pred=tune_df.y_pred) )
confusion_matrix(y_true=tune_df.y,y_pred=tune_df.y_pred).T

I0731 17:06:46.436407 139869363488640 estimator.py:1145] Calling model_fn.
I0731 17:06:46.475998 139869363488640 estimator.py:1147] Done calling model_fn.
I0731 17:06:46.558421 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:06:46.567446 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-12000
I0731 17:06:46.601592 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:06:46.610468 139869363488640 session_manager.py:502] Done running local_init_op.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


best mse threshold =  0.0180919598788023
0.5302593659942363 0.4681933842239186


array([[227289,    209],
       [   163,    184]])

# Validation Set Performance

In [65]:
test_pred = ae.predict(sc.transform(test_X),batch_size=20000)

pred_mse = []
pred_X_clone = []

for tp in test_pred:
  pred_mse.append( tp['mse'] )
  pred_X_clone.append( tp['Z'] )

test_pred_X_clone = np.array(pred_X_clone)

vdf = pd.DataFrame({'mse':pred_mse,'y':test_y.tolist()})
vdf['y_pred'] = (vdf.mse >= best_thres).astype('int')
print( precision_score(y_true=vdf.y,y_pred=vdf.y_pred),recall_score(y_true=vdf.y,y_pred=vdf.y_pred) )
confusion_matrix(y_true=vdf.y,y_pred=vdf.y_pred).T


I0731 17:07:23.246297 139869363488640 estimator.py:1145] Calling model_fn.
I0731 17:07:23.286648 139869363488640 estimator.py:1147] Done calling model_fn.
I0731 17:07:23.371632 139869363488640 monitored_session.py:240] Graph was finalized.
I0731 17:07:23.379802 139869363488640 saver.py:1280] Restoring parameters from /content/drive/My Drive/autoencoder_example/autoencoder_model_v1/model.ckpt-12000
I0731 17:07:23.413257 139869363488640 session_manager.py:500] Running local_init_op.
I0731 17:07:23.424233 139869363488640 session_manager.py:502] Done running local_init_op.


0.6027397260273972 0.4444444444444444


array([[56834,    55],
       [   29,    44]])